# Table of Contents
 <p><div class="lev1"><a href="#Import-data"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import data</a></div><div class="lev2"><a href="#School-District-Boundaries"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>School District Boundaries</a></div><div class="lev2"><a href="#City-Boundaries"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>City Boundaries</a></div><div class="lev2"><a href="#Sales"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Sales</a></div><div class="lev2"><a href="#Parcel-geolocation"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Parcel geolocation</a></div><div class="lev2"><a href="#Covariates"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Covariates</a></div><div class="lev1"><a href="#Projection"><span class="toc-item-num">2&nbsp;&nbsp;</span>Projection</a></div><div class="lev1"><a href="#Map-of-school-districts"><span class="toc-item-num">3&nbsp;&nbsp;</span>Map of school districts</a></div><div class="lev1"><a href="#Merge-sales,-covariates,-and-parcel-data"><span class="toc-item-num">4&nbsp;&nbsp;</span>Merge sales, covariates, and parcel data</a></div><div class="lev2"><a href="#Missing-data"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Missing data</a></div><div class="lev1"><a href="#Plotting-house-sales-in-Tucson"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plotting house sales in Tucson</a></div><div class="lev1"><a href="#Attach-school-district-to-each-sale"><span class="toc-item-num">6&nbsp;&nbsp;</span>Attach school district to each sale</a></div><div class="lev1"><a href="#Borders"><span class="toc-item-num">7&nbsp;&nbsp;</span>Borders</a></div><div class="lev2"><a href="#Example"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Example</a></div><div class="lev2"><a href="#Automate"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Automate</a></div><div class="lev1"><a href="#Export"><span class="toc-item-num">8&nbsp;&nbsp;</span>Export</a></div>

Merge the sales data with the parcel location data to get geocoded sales data, to be used to build our Spatial RDD example.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# plt.rc("figure", autolayout=True)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8.0, 6.0)

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import zipfile
from simpledbf import Dbf5
plot_dataframe = gpd.plotting.plot_dataframe

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


In [3]:
import shapely
from shapely.geometry import Point, LineString, MultiLineString, MultiPoint

# Import data

## School District Boundaries

In [4]:
schdistrs=gpd.read_file("national_data/Tiger/tlgdb_2015_a_us_school.gdb/a00000009.gdbtable")

## City Boundaries

In [5]:
cities=gpd.read_file("national_data/Place_2010Census_DP1/Place_2010Census_DP1.shp").to_crs(schdistrs.crs)

## Sales

In [6]:
tucson_sales_yearly=[]
for year in range(2011,2016):
    tucson_sales_yearly.append(pd.read_csv("Tucson_data/Sales/Sale%d.csv" % year,
        dtype={"SaleDate": str, "SequenceNum": str, "SalePrice": str}
    ))

In [7]:
tucson_sales

NameError: name 'tucson_sales' is not defined

## Parcel geolocation

In [ ]:
adparcel=gpd.read_file("Tucson_data/adparcel/adparcel.shp")
adparcel.set_index("PARCEL", inplace=True)

In [ ]:
print(adparcel.columns)

## Covariates

In [ ]:
mas10=Dbf5("Tucson_data/Residential/MAS2010.DBF").to_dataframe()
mas11=Dbf5("Tucson_data/Residential/MAS2011.DBF").to_dataframe()
mas10.head()

In [ ]:
mas14=pd.read_csv("Tucson_data/Residential/Mas14.csv", dtype={"VALUATIONC":"S3"})
mas16=pd.read_csv("Tucson_data/Residential/Mas16.csv", dtype={"VALUATIONC":"S3"})
mas17=pd.read_csv("Tucson_data/Residential/Mas17.csv", dtype={"VALUATIONC":"S3"})

In [ ]:
print(len(set(mas10.PARCEL).intersection(set(mas16.PARCEL))), "parcels in both 2010 and 2016")
print(len(set(mas10.PARCEL).difference(set(mas16.PARCEL))), "parcels in 2010 not 2016")
print(len(set(mas16.PARCEL).difference(set(mas10.PARCEL))), "parcels in 2016 not 2010")
print(len(set(mas14.PARCEL).difference(set(mas16.PARCEL))), "parcels in 2014 not 2016")
print(len(set(mas17.PARCEL).difference(set(mas16.PARCEL))), "parcels in 2017 not 2016")

In [ ]:
mas16.columns

Some columns are integer levels of factors that have corresponding labels, which are available online. Let's replace those columns with categorical columns to make data manipulation and interpretation more intuitive.

In [ ]:
# from http://www.asr.pima.gov/downloads/pages/layout.aspx?tbl=MAS&year=2015
mas16_codes = {
    "Class": dict(enumerate(["LM","R-1","R-2","R-3","R-4","R-5","R-6"],0)),
    "Quality": dict(enumerate(["Minimum","Fair","Good","Excellent"],1)),
    "Walls": dict(enumerate(["Framed Wood","Framed Block","8 inch Painted","8 inch Stucco",
              "Brick","Stone","SlumpBlock","Adobe","Other"], 0)),
    "Roof": dict(enumerate(["Wood","Asphalt","Asbestos","Built Up",
                            "Tile","Slate","Metal","Prepared Roll","Other"], 0)),
    "Heat": dict(enumerate(["Gravity","Forced","Steam","Hot Water","Radiant","Floor Furnance" # [sic]
             ,"Wall Furnance", "Electric Panel", "Other", "None"], 0)),
    "Cool": {0: "Refrigeration",1:"Evaporative",2:"Wall",9:"None"},
    "Patio": {1:"Slab",3:"Covered",6:"Covered Slab (both)",9:"None"},
#     "Condition": dict(enumerate(["Minimum","Fair","Good","Excellent"],1)), # doesn't seem to be accurate
    "Garage": {1:"Garage",3:"Carport",6:"Garage & Carport (both)",9:"None"},
}

In [ ]:
set(mas16["CONDITION"])

In [ ]:
for key, labels_dict in mas16_codes.items():
    print(key)
    col=mas16[key.upper()]
    cat=pd.Categorical(col)
    new_labels=[labels_dict[k] for k in cat.categories]
    cat.rename_categories(new_labels, inplace=True)
    mas16[key.upper()] = cat
for key, labels_dict in mas16_codes.items():
    print(key)
    col=mas17[key.upper()]
    cat=pd.Categorical(col)
    new_labels=[labels_dict[k] for k in cat.categories]
    cat.rename_categories(new_labels, inplace=True)
    mas17[key.upper()] = cat
print(mas16["GARAGE"].value_counts())
print(mas17["GARAGE"].value_counts())

In [ ]:
masAdds16=pd.read_csv("Tucson_data/Additions/MasAdds16.csv")
masAdds16.columns

In [ ]:
# from http://www.asr.pima.gov/downloads/pages/layout.aspx?tbl=MAS&year=2015
masAdds16_codes = {
    "SfrCondo": {"S": "Single Family Residence"},
    "Class": dict(enumerate(["LM","R-1","R-2","R-3","R-4","R-5","R-6"],0)),
    "Walls": dict(enumerate(["Framed Wood","Framed Block","8 inch Painted","8 inch Stucco",
              "Brick","Stone","SlumpBlock","Adobe","Other"], 0)),
    "Roof": dict(enumerate(["Wood","Asphalt","Asbestos","Built Up",
                            "Tile","Slate","Metal","Prepared Roll","Other"], 0)),
    "Heat": dict(enumerate(["Gravity","Forced","Steam","Hot Water","Radiant","Floor Furnance" # [sic]
             ,"Wall Furnance", "Electric Panel", "Other", "None"], 0)),
    "Cool": {0: "Refrigeration",1:"Evaporative",2:"Wall",9:"None"},
    "Patio": {1:"Slab",3:"Covered",6:"Covered Slab (both)",9:"None"},
#     "Condition": dict(enumerate(["Minimum","Fair","Good","Excellent"],1)), # doesn't seem to be accurate
    "Garage": {1:"Garage",3:"Carport",6:"Garage & Carport (both)",9:"None"},
}

In [ ]:
for key, labels_dict in masAdds16_codes.items():
    print(key)
    col=masAdds16[key.upper()]
    cat=pd.Categorical(col)
    new_labels=[labels_dict[k] for k in cat.categories]
    cat.rename_categories(new_labels, inplace=True)
    masAdds16[key.upper()] = cat
masAdds16["GARAGE"].value_counts()

In [ ]:
land16=pd.read_csv("Tucson_data/Land/Land16.csv")
print(land16.size)
land16.head()

# Projection

Ultimately, our Gaussian Process code will need x and y planar coordinates, with which distances between two points can be calculated as $\sqrt{(x_1-x_2)^2+(y_1-y_2)^2}$. Therefore we can't use raw longitudes and latitudes, they first need to be projected onto a Euclidian plane. Arizona has its own [coordinate system](https://www.webcms.pima.gov/cms/One.aspx?portalId=169&pageId=28222) that is a good choice for this task, and is associated with the [EPSG code 26749](http://www.spatialreference.org/ref/epsg/26749/). All we need to do is convert the coordinate reference system (CRS) of all our dataframes to EPSG 26749.

In [ ]:
adparcel = adparcel.to_crs(epsg=26749)
schdistrs = schdistrs.to_crs(epsg=26749)
cities = cities.to_crs(epsg=26749)

In [ ]:
print(adparcel.ix[0, "geometry"])
print(adparcel.ix[1, "geometry"])

Let's take a quick moment to check that the distances returned by the geopandas library can also be obtained via the simple Euclidian distance formula.

In [ ]:
adparcel.geometry[0].distance(adparcel.geometry[1])

In [ ]:
np.sqrt((936436.4682794245-937315.5846172333)**2+(553187.1136988925-552895.5028526088)**2)

# Map of school districts

In [ ]:
def city_schdistrs(city):
    intersect = schdistrs[schdistrs.intersects(city)]
    areas = intersect.intersection(city).area
    # throw out districts that aren't at least a thousandth of the city's surface area
    nonzero_area = areas > (city.area/1000)
    return intersect[nonzero_area] 
    
def simplename(s):
    return (s
        .replace("District","")
        .replace("School","")
        .replace("Community","")
        .replace("Independent","")
        .replace("Unified","")
        .strip()
    )
def plot_city(city):
    plot_dataframe(city, color="white", linewidth=3)
    old_xlim = plt.xlim()
    old_ylim = plt.ylim()
    schdistrs = city_schdistrs(city.geometry.values[0])
    plot_dataframe(schdistrs, ax=plt.gca())
    plt.xlim(old_xlim[0]-0.1,old_xlim[1]+0.1)
    plt.ylim(old_ylim[0]-0.1,old_ylim[1]+0.1)

    for row in schdistrs.iterrows():
        name = row[1].NAME
        geom = row[1].geometry
        x = geom.centroid.x
        y = geom.centroid.y
        plt.text(np.clip(x,*plt.xlim()), np.clip(y,*plt.ylim()), simplename(name), horizontalalignment='center')

In [ ]:
tucson=cities[cities.NAMELSAD10=="Tucson city"].iloc[0:1]
plot_city(tucson)

# Merge sales, covariates, and parcel data

In [ ]:
TC_sales_geocoded=adparcel.join(tucson_sales, how="right")

Remove sales with no location (only 8311 sales are removed out of 114343, so no big deal).

In [ ]:
notnan = np.logical_not(TC_sales_geocoded.geometry.isnull().values)
TC_sales_notnan = TC_sales_geocoded[notnan]
pd.Series(notnan).value_counts()

In [ ]:
in_tucson = TC_sales_notnan.within(tucson.geometry.values[0])

In [ ]:
TC_sales_Tucson = TC_sales_notnan[in_tucson]
TC_sales_Tucson=TC_sales_Tucson.to_crs(tucson.crs)

Now merge in the covariates.

In [ ]:
TC_sales_Tucson=TC_sales_Tucson.join(mas17.set_index("PARCEL"), how="left")
TC_sales_Tucson=TC_sales_Tucson.join(masAdds16.set_index("PARCEL"), how="left", rsuffix="Adds")

## Missing data

**Bad news:** a quarter of sales don't have associates square footage data.

In [ ]:
pd.Series(np.isnan(TC_sales_Tucson.SQFT.values)).value_counts()

In [ ]:
print(np.isnan(TC_sales_Tucson.SQFT).mean(), "<- fraction of missing values")

In [ ]:
print(len(set(in_tucson[in_tucson.values].index.values).difference(set(mas16.PARCEL))), 
    "Tucson parcels missing in `mas16` dataset")
print(len(set(in_tucson[in_tucson.values].index.values).difference(set(mas17.PARCEL))), 
    "Tucson parcels missing in `mas17` dataset")

How can 9615 sales have missing square footage, but only 5512 parcels be missing from the `mas17` dataset? Well, for one thing, some parcels are repeatedly sold and account for many sales.

In [ ]:
import collections
c=collections.Counter(in_tucson[in_tucson].index.values)
c.most_common(10)

Let's have a closer look at sales with missing square footage. First of all many are commercial properties or vacant land, so will be filtered out when we focus on residential buildings.

In [ ]:
nosqft=np.isnan(TC_sales_Tucson.SQFT.values)
TC_sales_Tucson.iloc[nosqft,:].PropertyType.value_counts()

Let's have a closer look at single family houses:

In [ ]:
TC_sales_Tucson[nosqft & (TC_sales_Tucson.PropertyType=="Single Family")]

In [ ]:
pd.set_option("display.max_columns",200)
pd.set_option("display.max_info_columns",200)
pd.set_option("display.max_rows",200)

The first house listed with parcel ID 10512058B has details available at http://www.asr.pima.gov/links/frm_AdvancedSearch_v2.aspx?search=Parcel . Why isn't it in in mas17? It's also not in the land dataset:

In [ ]:
land16[land16.PARCEL=="10512058B"]

It turns out the problem was that the parcel was in lowercase in the mas datasets! Here is the data:

In [ ]:
mas17[mas17.PARCEL=="10512058b"]

But still no land data:

In [ ]:
land16.PARCEL[land16.PARCEL>="10512058"].sort_values()[:1]

In [ ]:
TC_sales_Tucson = TC_sales_notnan[in_tucson]
TC_sales_Tucson=TC_sales_Tucson.to_crs(tucson.crs)
mas17.PARCEL=mas17.PARCEL.str.upper()
mas17.PARCEL=mas17.PARCEL.str.strip()
land16.PARCEL=land16.PARCEL.str.upper()
land16.PARCEL=land16.PARCEL.str.strip()
masAdds16.PARCEL=masAdds16.PARCEL.str.upper()
masAdds16.PARCEL=masAdds16.PARCEL.str.strip()
TC_sales_Tucson=TC_sales_Tucson.join(mas17.set_index("PARCEL"), how="left")
TC_sales_Tucson=TC_sales_Tucson.join(masAdds16.set_index("PARCEL"), how="left", rsuffix="Adds")
TC_sales_Tucson=TC_sales_Tucson.join(land16.set_index("PARCEL"), how="left", rsuffix="Land")

In [ ]:
nosqft=np.isnan(TC_sales_Tucson.SQFT.values)
TC_sales_Tucson.iloc[nosqft,:].PropertyType.value_counts()

Less progress than hoped! Still 339 (down from 353) single family homes with no square footage data. Let's look at the next one down.

In [ ]:
TC_sales_Tucson[nosqft & (TC_sales_Tucson.PropertyType=="Single Family")]

Let's investigate parcel 106011260. No residential covariates, but we do have land information for this one:

In [ ]:
land16[land16.PARCEL=="106011260"]

Looking up the property on [asr.pima.gov](http://www.asr.pima.gov/links/frm_AdvancedSearch_v2.aspx?search=Parcel), residential characteristics aren't available there either, instead there is data on commercial characteristics. Maybe this property is being used commercially? This is odd, as it's also labeled as a primary residence. Furthermore, we have a sale recorded in October 2013 for \$86,900, described as a “Sale to or from a government agency”. In 2006 the same house sold for $205,000, so the low sale price is odd. It's not marked as a “partial interest” either, which I think might mean that only part of the property was purchased. It's also not marked as a “personal property”, but then again that's the case for the vast majority of sales, so I don't think that's hugely significant.

In [ ]:
TC_sales_Tucson.ValidationDescription.value_counts()

In [ ]:
TC_sales_Tucson.PartialInterest.value_counts()

In [ ]:
TC_sales_Tucson.PersonalProperty.value_counts()

# Plotting house sales in Tucson

In [ ]:
plot_city(tucson)
plot_dataframe(TC_sales_Tucson, ax=plt.gca(), color="black", alpha=0.2)

Each dot represents the sale of a property in 2015. The borders between school districts in the North-West part of the city look particularly promising. The border between Tucson Unified and Sunnyside Unified could also work. The long straight border between Tucson Unified and Vail Unified is unfortunately only populated on one side.

# Attach school district to each sale

In [ ]:
tucson_schdistrs=city_schdistrs(tucson.geometry.values[0])
tucson_schdistrs

In [ ]:
def get_ind(zones, point):
    ind=np.where(zones.contains(point))[0]
    if len(ind)==0:
        return None
    elif len(ind)>1:
        return None#np.argmin(zones.iloc[ind].area)
    return ind[0]

In [ ]:
idistr=[get_ind(tucson_schdistrs, house) for house in TC_sales_Tucson.geometry]

In [ ]:
sum([i is None for i in idistr])

In [ ]:
TC_sales_Tucson["SchDistr"]=tucson_schdistrs.NAME.values[idistr]

# Borders

## Example

In [ ]:
d1=tucson_schdistrs.geometry.values[0]
d2=tucson_schdistrs.geometry.values[1]

In [ ]:
border=d1.intersection(d2)
assert isinstance(border, MultiLineString)

In [ ]:
len(border.geoms)

In [ ]:
max([l.length for l in border])

In [ ]:
MultiPoint([border.interpolate(x, normalized=True) for x in np.linspace(0,1.0,100)])

In [ ]:
shapely.ops.linemerge(border.geoms)

## Automate

In [ ]:
import itertools

In [ ]:
ndistr = tucson_schdistrs.shape[0]
distr1_ls=[]
distr2_ls=[]
sentinels_ls=[]
borders_ls=[]

for i,j in itertools.combinations(range(ndistr),2):
    di = tucson_schdistrs.geometry.values[i]
    dj = tucson_schdistrs.geometry.values[j]
    distr_i = tucson_schdistrs.NAME.values[i]
    distr_j = tucson_schdistrs.NAME.values[j]
    border = di.intersection(dj)
    if not border.geoms:
        continue
#     assert isinstance(border, MultiLineString)
    merged_border = shapely.ops.linemerge(border)
    sentinels = MultiPoint([border.interpolate(x, normalized=True) for x in np.linspace(0,1.0,100)])
    sentinels_ls.append(sentinels)
    sentinels_ls.append(sentinels)
    borders_ls.append(merged_border)
    borders_ls.append(merged_border)
    distr1_ls.append(distr_i)
    distr1_ls.append(distr_j)
    distr2_ls.append(distr_j)
    distr2_ls.append(distr_i)

In [ ]:
borders = gpd.GeoDataFrame({"SchoolDistrict1":distr1_ls, "SchoolDistrict2": distr2_ls, "geometry": borders_ls})
sentinels = gpd.GeoDataFrame({"SchoolDistrict1":distr1_ls, "SchoolDistrict2": distr2_ls, "geometry": sentinels_ls})

In [ ]:
plot_dataframe(borders)
plt.title("Borders between school districts")

# Export

In [ ]:
TC_sales_Tucson["X_PRJ"] = [p.x for p in TC_sales_Tucson.geometry.values]
TC_sales_Tucson["Y_PRJ"] = [p.y for p in TC_sales_Tucson.geometry.values]

In [ ]:
TC_sales_Tucson.to_csv("Tucson_data/processed/Tucson_sales.csv")

In [ ]:
borders.to_file('Tucson_data/processed/SchoolDistrict_borders/SchoolDistrict_borders.shp', driver='ESRI Shapefile')

In [ ]:
sentinels.to_file('Tucson_data/processed/SchoolDistrict_borders/SchoolDistrict_sentinels.shp', driver='ESRI Shapefile')

In [ ]:
with open("Tucson_data/processed/SchoolDistrict_borders/SchoolDistrict_borders.json", "w") as f:
    f.write(borders.to_json())

In [ ]:
with open("Tucson_data/processed/SchoolDistrict_borders/SchoolDistrict_sentinels.json", "w") as f:
    f.write(sentinels.to_json())